In [99]:
import pandas as pd
import re
import networkx as nx
import copy
import sqlite3
from sqlalchemy import create_engine

### Generazione della rete di menzioni+retweet

In [3]:
df = pd.read_parquet("All_Tweets.parquet.gz", engine='fastparquet')

In [6]:
df.head(3)

,lang,text,author_id,created_at,id,public_metrics_retweet_count,public_metrics_reply_count,public_metrics_like_count,public_metrics_quote_count,withheld_copyright,withheld_country_codes,withheld_scope
0,en,RT @EU_Commission: Tomorrow it's the start of ...,91875994,2019-10-07T02:37:25.000Z,1181035773514174464,101,0,0,0,NaN,None,None
1,en,RT @BBCNewsNI: Ever wonder what it takes to be...,91875994,2019-10-07T02:37:17.000Z,1181035740412698624,13,0,0,0,NaN,None,None
2,en,RT @LynetteFay: My mother brought us to work w...,91875994,2019-10-07T02:35:10.000Z,1181035206670786560,10,0,0,0,NaN,None,None


In [7]:
df_en = df[df.lang == 'en']

In [8]:
author_ids = list(df_en['author_id'].unique())

In [9]:
len(author_ids)

3958

In [12]:
regex=re.compile(r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9_]+)")

def extract_mentions(text):
    return regex.findall(text.lower())

In [ ]:
df_en['mentions'] = df_en['text'].apply(extract_mentions)
#df_en[['text', 'mentions']]

In [15]:
# sostituisci gli id con gli screen_name

associazione_id_screen_name = {}
file = open('ASSOCIAZIONE_TOTALE_IDS_SCREEN_NAMES.csv')
while 1:
    line = file.readline()
    if not line:
        break
    id = line.split(',')[0]
    screen_name = line.split(',')[1][:-1].lower()
    associazione_id_screen_name[id] = screen_name

In [16]:
# carica gli id di account non piu' esistenti

ids_mancanti=[]
file = open('ids_mancanti.txt')
while 1:
    line = file.readline()
    if not line:
        break
    ids_mancanti.append(int(line[:-1]))

print(ids_mancanti)

[136573214, 4072525400, 950343771752947713]


In [18]:
def convert_id_to_screen_name(id):
    try:
        return associazione_id_screen_name[id]
    except Exception as e:
        if int(str(e).replace("'",'')) in ids_mancanti:
            return 'DELETED_ACCOUNT'
        else:
            raise
        

df_en['author_screen_name'] = df_en['author_id'].apply(convert_id_to_screen_name)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [19]:
# esplode le liste di menzioni creando una riga per ogni elemento della lista e rimuove le righe con Nan 
# (appaiono quando le liste sono vuote)
# vedi: https://stackoverflow.com/questions/45846765/efficient-way-to-unnest-explode-multiple-list-columns-in-a-pandas-dataframe

df_en_edgelist = df_en[['author_screen_name', 'mentions']].set_index(['author_screen_name']).apply(pd.Series.explode).reset_index().dropna().sort_values('author_screen_name')

In [21]:
# crea due nuove colonne src-trg seguendo il criterio per cui la stringa di src venga prima di quella di trg
# (rispetto all'ordine alfabetico); questo serve per poter poi calcolare i pesi dei link nel caso non diretto

def crea_edgelist_ordinata_src(row):
    return sorted([row[0], row[1]])[0]
def crea_edgelist_ordinata_trg(row):
    return sorted([row[0], row[1]])[1]
        

df_en_edgelist['src'] = pd.DataFrame(df_en_edgelist.apply(crea_edgelist_ordinata_src, axis=1))
df_en_edgelist['trg'] = pd.DataFrame(df_en_edgelist.apply(crea_edgelist_ordinata_trg, axis=1))

In [22]:
# CALCOLA IL PESO DEI LINK

df_en_weighted_edgelist = df_en_edgelist.groupby(['src', 'trg']).agg(['count']).reset_index()
df_en_weighted_edgelist

,src,trg,author_screen_name,mentions
,,,count,count
0,100erneuerbar,agoraew,1,1
1,100erneuerbar,ahmadmansour__,1,1
2,100erneuerbar,anneapplebaum,1,1
3,100erneuerbar,antonioguterres,2,2
4,100erneuerbar,aukehoekstra,1,1
...,...,...,...,...
807027,zpp_eu,zppn,1,1
807028,zpp_eu,zppnetpl,31,31
807029,zpp_eu,zsoltdarvas,13,13


In [23]:
df_en_weighted_edgelist['weight'] = df_en_weighted_edgelist['author_screen_name']['count']

In [24]:
df_en_weighted_edgelist = df_en_weighted_edgelist.drop(columns=['author_screen_name', 'mentions'])
df_en_weighted_edgelist

/home/vincenzo/.local/lib/python3.7/site-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,src,trg,weight
,,,
0,100erneuerbar,agoraew,1
1,100erneuerbar,ahmadmansour__,1
2,100erneuerbar,anneapplebaum,1
3,100erneuerbar,antonioguterres,2
4,100erneuerbar,aukehoekstra,1
...,...,...,...
807027,zpp_eu,zppn,1
807028,zpp_eu,zppnetpl,31
807029,zpp_eu,zsoltdarvas,13


In [28]:
G = nx.from_pandas_edgelist(df_en_weighted_edgelist, 'src', 'trg', ['weight'])

In [29]:
len(list(G.nodes()))

355238

In [30]:
len(list(G.edges()))

807032

In [32]:
df_en_directed_weighted_edgelist = df_en_edgelist.groupby(['author_screen_name', 'mentions']).agg(['count']).reset_index()


In [33]:
df_en_directed_weighted_edgelist['weight'] = df_en_directed_weighted_edgelist['src']['count']


In [35]:
df_en_directed_weighted_edgelist = df_en_directed_weighted_edgelist.drop(columns=['src', 'trg'])


/home/vincenzo/.local/lib/python3.7/site-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [36]:
DG = nx.from_pandas_edgelist(df_en_directed_weighted_edgelist, 'author_screen_name', 'mentions', ['weight'], create_using=nx.DiGraph())

In [37]:
len(list(DG.nodes()))

355238

In [38]:
len(list(DG.edges()))

810220

In [39]:
scc = nx.strongly_connected_components(DG)

len_list = []
for i in scc:
    len_list.append(len(i))
len_list.sort(reverse=True)
print(len_list[:10])
print(len(len_list))

[2094, 8, 6, 4, 3, 3, 3, 2, 2, 2]
353090


In [42]:
scc = nx.strongly_connected_components(DG)

scc_max = []
for a in scc:
#     print(len(a))
    if len(a)>len(scc_max):
        scc_max = a
        
nodes = copy.deepcopy(DG.nodes())

for n in nodes:
    if n not in scc_max:
        DG.remove_node(n)

In [43]:
len(list(DG.nodes()))

2094

In [44]:
len(list(DG.edges()))

35952

In [45]:
# rimuovo i nodi disconnessi anche dalla rete non diretta

nodes = copy.deepcopy(G.nodes())

for n in nodes:
    if n not in scc_max:
        G.remove_node(n)

In [46]:
len(list(G.nodes()))

2094

In [47]:
len(list(G.edges()))

32824

### Generazione della rete di Friendship

In [56]:
df_FRIENDSHIP = pd.read_csv('links_tot_EN.csv')
df_FRIENDSHIP['Source'] = df_FRIENDSHIP['Source'].apply(lambda x : x.lower())
df_FRIENDSHIP['Target'] = df_FRIENDSHIP['Target'].apply(lambda x : x.lower())
df_FRIENDSHIP['Link'] = df_FRIENDSHIP['Source']+'----'+df_FRIENDSHIP['Target']
df_FRIENDSHIP['InverseLink'] = df_FRIENDSHIP['Target']+'----'+df_FRIENDSHIP['Source']

In [57]:
G_FRIENDSHIP = nx.from_pandas_edgelist(df_FRIENDSHIP, 'Source', 'Target', create_using=nx.Graph())

In [58]:
len(list(G_FRIENDSHIP.nodes()))

4954

In [59]:
len(list(G_FRIENDSHIP.edges()))

13806

### Studio della sovrapposizione dei link

In [60]:
FRIENDSHIP_links = set(df_FRIENDSHIP['Link']).union(set(df_FRIENDSHIP['InverseLink']))
FRIENDSHIP_links

{'euenergyweek----annikaahtonen',
 'dekgreenhome----greencogeu',
 'theloudbloke----marinecng',
 'martadassu----raffaelvincenti',
 'errinnetwork----energycities',
 'awea----cogeneurope',
 'naftogazukraine----mintyteaboy',
 'bboxx_hq----greenunivers',
 'energy4europe----eucouncilpress',
 'eu_env----jorgensenjuul',
 'actrisri----eu_env',
 'wmbtweets----koencoppenholle',
 'nielsvanderaar----iogp_news',
 'jehlandclearing----egc_conference',
 'greencogeu----helloheatpumps',
 'ppfenergia----smartgrid_updt',
 'alorin_fr----chrisdebien',
 'chrisnelder----argusmedia',
 'orsted----vestager',
 'l_dejonge----willwilkesbbg',
 'cecberg----eucouncil',
 'gmaxtidalenergy----jerico_next',
 'worldwatch----amccloat',
 'climate_rescue----toconnordublin',
 'greencogeu----california_iso',
 'eu_env----ep_budgets',
 'eu_env----europeanchamber',
 'bellitaliasrl----casaimpattozero',
 'greenafricadir----sustain_cities',
 'euinaus----350australia',
 'eusmartcities----energy_post',
 'netzerolive----iigccnews',
 'lum

In [61]:
len(FRIENDSHIP_links)

27612

In [81]:
MENTION_links = []
for l in list(G.edges()):
    MENTION_links.append(l[0]+'----'+l[1])
    MENTION_links.append(l[1]+'----'+l[0])
MENTION_links = set(MENTION_links)
MENTION_links

{'sachmal----eu_greencapital',
 'andreamorawski----eucouncilpress',
 'energy4europe----policyregi',
 'vestager----erek_eu',
 'europeenergy----europeenergy',
 'climatekic----reeemproject',
 'timmermanseu----ebc_smes',
 'rtds_group----bulc_eu',
 'heidihautala----euniclondon',
 'azoreseuoffice----europacinemas',
 'eu_commission----wefenergy',
 'actrisri----eu_env',
 'yannikouts----mfa_lu',
 'greenbiz----sh_bahramirad',
 'margschinas----cellinides',
 'fern_mar----eu_commission',
 'bulc_eu----kelsey_perlman',
 'gerdzuendorf----stylianideseu',
 'margschinas----refugees_eu',
 'eurocrowd----rescoopeu',
 'ep_president----slowfoodeurope',
 'earsc----nextgeoss',
 'johnbachtler----eurada_rdas',
 'causse_eu----ep_environment',
 'efsa_eu----kolossamarike',
 'entsog----arera_it',
 'ipcc_ch----climateurope',
 'ipcc_ch----searica_itg',
 'energy4europe----gdcharter',
 'etoa----aci_europe',
 'ema_news----sandragallina',
 'riaeu----agatagostynska',
 'cedentists----epsa_online',
 'eumicon----ep_environment

In [82]:
len(MENTION_links)

63768

In [83]:
common_links = FRIENDSHIP_links.intersection(MENTION_links)
common_links

{'dhcplus----etipsnet',
 'eubionet1----eu_env',
 'euenergyweek----jagopri',
 'energy4europe----eucouncilpress',
 'eu_env----jorgensenjuul',
 'actrisri----eu_env',
 'energy4europe----eusmartcities',
 'eu_managenergy----efiees',
 'ocean_climate----searica_itg',
 'naftogazukraine----eucopresident',
 'acglobalenergy----eu_commission',
 'eurima_mw----eu_env',
 'argusmedia----epure_ethanol',
 'esa----eu_env',
 'monicafrassoni----kyoto_club',
 'egec_geothermal----ehiassociation',
 'euinlebanon----eu_env',
 'eu_env----keti_medarova',
 'lifewatcheric----eu_env',
 'kjchalvatzis----nobelgrid',
 'euenergyweek----europarl_en',
 'clnruk----centricaplc',
 'ceciliawikstrom----eu_env',
 'eu_env----clairecouet',
 'eu_env----h2020elobster',
 'charm_eu----eu_env',
 'eu_commission----edfenergyex',
 'mac_europa----eu_eurostat',
 'eubac----greencogeu',
 'eu_env----euroargoeric',
 'clean_energy_sc----mackaymiller',
 'epp----mac_europa',
 'ehnheart----eu_env',
 'acglobalenergy----irinamarkinaeu',
 'joeeg4a----

In [84]:
len(common_links)

3058

### Studio della rete di sovrapposizione

In [87]:
nodi_di_sovrapposizione = set()
for l in common_links:
#     print(l.split('----'))
    src = l.split('----')[0]
    trg = l.split('----')[1]
    nodi_di_sovrapposizione.add(src)
    nodi_di_sovrapposizione.add(trg)

In [89]:
len(nodi_di_sovrapposizione)

966

In [92]:
df_desc = pd.read_csv('DescrizioneUtenti_e_AppartenenzaAllaReteDiMenzioniOFriendship.csv')[["screen_name","net_friendship","net_mentions","description"]]

In [93]:
df_desc

,screen_name,net_friendship,net_mentions,description
0,enelgroupit,True,True,L'azienda che sta cambiando il volto dell'energia
1,iccgov_climate,True,True,Disseminating research on climate policy and g...
2,ergnow,True,False,Canale ufficiale del primo operatore #eolico i...
3,sciencetargets,True,True,Enabling businesses to take swift climate acti...
4,edisonnews,True,True,Diventiamo l’#energiachecambiatutto. Anche con...
...,...,...,...,...
4765,maelysdervenn,True,False,AbFab's Eddie Moonson would have managed brexi...
4766,_beyondthegrid,True,False,living off grid. Trying to create a sustainabl...
4767,friendsofjef,True,False,Friends of JEF is the alumni organisation for ...
4768,jewisheritage,True,True,The European Association for the Preservation ...


In [97]:
df_desc_sovrapposizione = df_desc[df_desc['screen_name'].isin(nodi_di_sovrapposizione)]
df_desc_sovrapposizione['user_classification'] = '?'
df_desc_sovrapposizione

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,screen_name,net_friendship,net_mentions,description,user_classification
1,iccgov_climate,True,True,Disseminating research on climate policy and g...,?
3,sciencetargets,True,True,Enabling businesses to take swift climate acti...,?
5,ge_power,True,True,"We provide technology, solutions and services ...",?
6,eni,True,True,"+innovation, +efficiency and +accessibility fo...",?
8,eu_env,True,True,The official account for @EU_Commission Direct...,?
...,...,...,...,...,...
4724,plouislempereur,True,True,Deputy Head of Division Human Resources Policy...,?
4741,epsa_online,True,True,European Pharmaceutical Students' Association ...,?
4749,eucitsciproject,True,True,The new European #CitizenScience online #platf...,?
4750,slowfoodeurope,True,True,European policies affect us all! Brussels base...,?


In [98]:
df_desc_sovrapposizione.to_csv('DescrizioneUtenti_e_AppartenenzaAllaReteDiMenzioniOFriendship_SolaReteDiSovrapposizione.csv')

In [100]:
engine = create_engine('sqlite:///DescrizioneUtenti_SolaReteDiSovrapposizione.db', echo=True)
sqlite_connection = engine.connect()

2021-07-12 14:50:05,712 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-07-12 14:50:05,713 INFO sqlalchemy.engine.base.Engine ()
2021-07-12 14:50:05,714 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-07-12 14:50:05,715 INFO sqlalchemy.engine.base.Engine ()


In [101]:
sqlite_table = "DescrizioneUtenti"
df_desc_sovrapposizione.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-07-12 14:50:28,970 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("DescrizioneUtenti")
2021-07-12 14:50:28,971 INFO sqlalchemy.engine.base.Engine ()
2021-07-12 14:50:28,972 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("DescrizioneUtenti")
2021-07-12 14:50:28,972 INFO sqlalchemy.engine.base.Engine ()
2021-07-12 14:50:28,976 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "DescrizioneUtenti" (
	"index" BIGINT, 
	screen_name TEXT, 
	net_friendship BOOLEAN, 
	net_mentions BOOLEAN, 
	description TEXT, 
	user_classification TEXT, 
	CHECK (net_friendship IN (0, 1)), 
	CHECK (net_mentions IN (0, 1))
)


2021-07-12 14:50:28,976 INFO sqlalchemy.engine.base.Engine ()
2021-07-12 14:50:28,984 INFO sqlalchemy.engine.base.Engine COMMIT
2021-07-12 14:50:28,984 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_DescrizioneUtenti_index" ON "DescrizioneUtenti" ("index")
2021-07-12 14:50:28,984 INFO sqlalchemy.engine.base.Engine ()
2021-07-12 14:50:28,990 INFO sqlalchemy.eng

### creazione del dataset della rete di non sovrapposizione

In [102]:
df_desc_NONsovrapposizione = df_desc[~df_desc['screen_name'].isin(nodi_di_sovrapposizione)]

In [103]:
df_desc_NONsovrapposizione

,screen_name,net_friendship,net_mentions,description
0,enelgroupit,True,True,L'azienda che sta cambiando il volto dell'energia
2,ergnow,True,False,Canale ufficiale del primo operatore #eolico i...
4,edisonnews,True,True,Diventiamo l’#energiachecambiatutto. Anche con...
7,qatarpetroleum,True,False,Welcome to the official Twitter account of @Qa...
13,jjkirton,True,False,"Director, G7 Research Group, G20 Research Grou..."
...,...,...,...,...
4765,maelysdervenn,True,False,AbFab's Eddie Moonson would have managed brexi...
4766,_beyondthegrid,True,False,living off grid. Trying to create a sustainabl...
4767,friendsofjef,True,False,Friends of JEF is the alumni organisation for ...
4768,jewisheritage,True,True,The European Association for the Preservation ...


In [104]:
df_desc_NONsovrapposizione[df_desc_NONsovrapposizione['net_friendship']==True].count()

screen_name       3804
net_friendship    3804
net_mentions      3804
description       3782
dtype: int64

In [105]:
df_desc_NONsovrapposizione[df_desc_NONsovrapposizione['net_mentions']==True].count()

screen_name       1128
net_friendship    1128
net_mentions      1128
description       1126
dtype: int64

In [106]:
df_desc_sovrapposizione[df_desc_sovrapposizione['net_friendship']==True].count()

screen_name            966
net_friendship         966
net_mentions           966
description            963
user_classification    966
dtype: int64

In [107]:
df_desc_sovrapposizione[df_desc_sovrapposizione['net_mentions']==True].count()

screen_name            966
net_friendship         966
net_mentions           966
description            963
user_classification    966
dtype: int64

In [108]:
1128/3804

0.29652996845425866

In [ ]:
"""
RETE DI SOVRAPPOSIZIONE: TUTTI I NODI APPARTENGONO SIA ALLA RETE DI MENZIONI/RETWEET CHE ALLA RETE DI FRIENDSHIP

RETE DI NON SOVRAPPOSIZIONE: TUTTI I NODI APPARTENGONO ALLA RETE DI FRIENDSHIP; 
                             DI QUESTI, CIRCA IL 30% (1128 SU 3804) APPARTENGONO ANCHE ALLA RETE DI MENZIONI/RETWEET
"""

In [110]:
df_desc_NONsovrapposizione.to_csv('DescrizioneUtenti_e_AppartenenzaAllaReteDiMenzioniOFriendship_SolaReteDi-NON-Sovrapposizione.csv')